In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor

from tensorflow.keras.models import load_model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

%matplotlib inline

### YOLO Box Filtering  
Filters predicted bounding boxes based on confidence scores and a threshold.

In [ ]:

def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold=.6):
    box_scores = box_confidence * box_class_probs  # Combine confidence and class probabilities
    box_classes = tf.argmax(box_scores, axis=-1)  # Get index of highest scoring class
    box_class_scores = tf.reduce_max(box_scores, axis=-1)  # Get highest class score
    filtering_mask = box_class_scores >= threshold  # Create mask for scores above threshold
    scores = tf.boolean_mask(box_class_scores, filtering_mask)  # Filter scores
    boxes = tf.boolean_mask(boxes, filtering_mask)  # Filter boxes
    classes = tf.boolean_mask(box_classes, filtering_mask)  # Filter classes
    return scores, boxes, classes  # Return filtered results

# Intersection over Union (IoU) Calculation  
Computes the IoU metric between two bounding boxes (format: x1, y1, x2, y2).

In [ ]:
def iou(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1  # Unpack box1 coordinates
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2  # Unpack box2 coordinates

    xi1 = max(box1_x1, box2_x1)  # Intersection x1 (left)
    yi1 = max(box1_y1, box2_y1)  # Intersection y1 (top)
    xi2 = min(box1_x2, box2_x2)  # Intersection x2 (right)
    yi2 = min(box1_y2, box2_y2)  # Intersection y2 (bottom)
    inter_width = max(0, xi2 - xi1)  # Intersection width (0 if no overlap)
    inter_height = max(0, yi2 - yi1)  # Intersection height (0 if no overlap)
    inter_area = inter_width * inter_height  # Intersection area

    box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)  # Area of box1
    box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)  # Area of box2
    union_area = box1_area + box2_area - inter_area  # Union area

    iou = inter_area / union_area if union_area != 0 else 0  # Compute IoU (avoid division by 0)
    return iou  # Return Intersection over Union value

### YOLO Non-Max Suppression  
Filters overlapping boxes using class-wise NMS and keeps top `max_boxes` predictions.  

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    boxes = tf.cast(boxes, dtype=tf.float32)  # Convert boxes to float32
    scores = tf.cast(scores, dtype=tf.float32)  # Convert scores to float32

    nms_indices = []  # List to store selected indices
    classes_labels = tf.unique(classes)[0]  # Get unique class labels
    
    for label in classes_labels:  # Process each class separately
        filtering_mask = classes == label  # Create mask for current class
        
        boxes_label = tf.boolean_mask(boxes, filtering_mask)  # Filter boxes by class
        scores_label = tf.boolean_mask(scores, filtering_mask)  # Filter scores by class
        
        if tf.shape(scores_label)[0] > 0:  # If any boxes exist for this class
            nms_indices_label = tf.image.non_max_suppression(  # Apply NMS
                boxes_label,
                scores_label,
                max_boxes,
                iou_threshold=iou_threshold)

            selected_indices = tf.squeeze(tf.where(filtering_mask), axis=1)  # Get original indices
            nms_indices.append(tf.gather(selected_indices, nms_indices_label))  # Store selected indices

    nms_indices = tf.concat(nms_indices, axis=0)  # Combine indices from all classes
    
    scores = tf.gather(scores, nms_indices)  # Gather selected scores
    boxes = tf.gather(boxes, nms_indices)  # Gather selected boxes
    classes = tf.gather(classes, nms_indices)  # Gather selected classes
    
    # Sort by scores and return the top max_boxes
    sort_order = tf.argsort(scores, direction='DESCENDING').numpy()  # Get sort order
    scores = tf.gather(scores, sort_order[0:max_boxes])  # Keep top scores
    boxes = tf.gather(boxes, sort_order[0:max_boxes])  # Keep top boxes
    classes = tf.gather(classes, sort_order[0:max_boxes])  # Keep top classes

    return scores, boxes, classes  # Return filtered results (fixed typo from 'clases')

### YOLO Box Coordinates Conversion 

In [ ]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """Convert YOLO box predictions to bounding box corners."""
    box_mins = box_xy - (box_wh / 2.)  # Calculate bottom-left corner (xmin, ymin)
    box_maxes = box_xy + (box_wh / 2.)  # Calculate top-right corner (xmax, ymax)

    return tf.keras.backend.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])

###  YOLO Output Evaluation Processes raw YOLO model outputs into final detections with filtering and NMS

In [ ]:
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    
    
   # Retrieve outputs of the YOLO model
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions (convert boxes box_xy and box_wh to corner coordinates)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Perform Score-filtering with a threshold of score_threshold
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold=score_threshold)

    # Scale boxes back to original image shape
    boxes = scale_boxes(boxes, image_shape)

    # Perform Non-max suppression with maximum number of boxes set to max_boxes and a threshold of iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes, iou_threshold=iou_threshold)
   
    
    return scores, boxes, classes

### YOLO Model Initialization  
Loads class names, anchor boxes, and pre-trained YOLO model for object detection. 

In [ ]:
# Load YOLO model configuration files and initialize model
class_names = read_classes("model_data/coco_classes.txt")  # Load 80 COCO class names (person, car, etc.)
anchors = read_anchors("model_data/yolo_anchors.txt")  # Load 9 YOLO anchor boxes (predefined bounding box shapes)
model_image_size = (608, 608)  # Input image dimensions required by YOLO network

# Load pre-trained YOLO model (without compilation since we're using it for inference)
yolo_model = load_model("model_data/", compile=False)  # Original weights trained on COCO dataset

### YOLO Object Detection Prediction
Performs end-to-end object detection on an input image using YOLO

In [ ]:
def predict(image_file):
    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), "images/" + image_file))
    # Generate colors for drawing bounding boxes.
    colors = get_colors_for_classes(len(class_names))
    # Draw bounding boxes on the image file
    #draw_boxes2(image, out_scores, out_boxes, out_classes, class_names, colors, image_shape)
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=100)
    # Display the results in the notebook
    output_image = Image.open(os.path.join("out", image_file))
    imshow(output_image)

    return out_scores, out_boxes, out_classes

### Run YOLO Object Detection
Detects objects in 'test.jpg' and returns bounding boxes, scores, and class labels

In [ ]:
# Make prediction on test image
out_scores, out_boxes, out_classes = predict("test.jpg")

# Output will contain:
# - out_scores: Confidence scores for each detection (0-1)
# - out_boxes: Bounding box coordinates [y1, x1, y2, x2] in original image scale
# - out_classes: Class indices corresponding to class_names